In [1]:
import os
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

## 数据准备

In [2]:
import numpy as np
from datasets.dataset import load_boston
from model_selection.train_test_split import train_test_split

data=load_boston()
X=data.data
Y=data.target

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

# 为便于numpy矩阵乘法之间的维度维护，统一将Y转换成列向量，与一行一个样本对应
Y_train=Y_train.reshape((-1,1))
Y_test=Y_test.reshape((-1,1))

# print(X_train.shape,Y_train.shape)

数据$X$是一个$(n{\times}m)$的矩阵，每一行是一个样本，每一列代表一个特征：

In [3]:
n_sample=X_train.shape[0]
n_feature=X_train.shape[1]

## 粗略模型

模型表达式为：
$$
\hat{y}=wx+b
$$
在向量化操作时，其中的$y$、$w$、$x$都会被矩阵形式替代。为了便于与后期深度学习的一致性，设$X$的维度为$(n\_sample,n\_feature)$，设$W$的维度为$(n\_feature,n\_output)$，偏置系数$b$为单变量系数。模型可以写成：
$$
\hat{Y}=XW+b
$$

In [4]:
W = np.random.randn(n_feature).reshape((n_feature, 1))  # 权重，列向量
b = 1  # 偏置

Y_hat=np.dot(X_train, W)+b

# print(W.shape,Y_hat.shape)

模型的损失函数为：
$$
\begin{align}
L&=\sum\limits_{i=1}^n(y^{(i)}-\hat{y}^{(i)})^{2} \\
&=\frac{1}{n}(Y-\hat{Y})^{T}(Y-\hat{Y}) \\
\end{align}
$$
损失函数关于参数$W$与$b$的梯度可以求得：
$$
\begin{align}
\frac{\partial{L}}{\partial{W}}&=\frac{2}{n}[X^{T}(\hat{Y}-Y)] \\
\frac{\partial{L}}{\partial{b}}&=\frac{2}{n}[1,1,...,1]{\cdot}(\hat{Y}-Y) \\
\end{align}
$$

In [5]:
dW = 2 * X_train.T.dot(Y_hat - Y_train) / n_sample
db = 2 * np.sum(Y_hat - Y_train)/ n_sample

# print((Y_hat - Y_train).shape,dW.shape,db.shape)

参数的迭代更新公式：
$$
W:=W-{\alpha}\frac{\partial{L}}{\partial{W}}, \quad b:b-{\alpha}\frac{\partial{L}}{\partial{b}}
$$

In [6]:
max_iter=2000
alpha=0.000001        # 注意学习率过大会导致震荡，然后误差越来越大

for i in range(max_iter):
    Y_hat=np.dot(X_train, W)+b
    
    dW = 2 * X_train.T.dot(Y_hat - Y_train) / n_sample
    db = 2 * np.sum(Y_hat - Y_train)/ n_sample
    
    W = W - alpha * dW
    b = b - alpha * db

使用该模型分别对训练集与预测集做预测：

In [7]:
Y_pred_train=np.dot(X_train, W) + b
Y_pred_test=np.dot(X_test, W) + b

定义一个RMSE损失函数来评价模型的表现：

In [8]:
def RMSE(Y_true,Y_pred):
    return np.sum((Y_true-Y_pred)**2)**0.5/len(Y_true)

print(RMSE(Y_train,Y_pred_train),RMSE(Y_test,Y_pred_test))

0.6449138841214859 1.3965156320353282


模型简单打包：

In [9]:
def linear_reg(X,Y,alpha=0.000001,max_iter=2000):
    Y=Y.reshape((-1,1))
    
    n_sample=X.shape[0]
    n_feature=X.shape[1]
    
    W = np.random.randn(n_feature).reshape((n_feature,1))  # 权重
    b = 1  # 偏置
    
    for i in range(max_iter):
        Y_hat=np.dot(X, W)+b

        dW = 2 * X.T.dot(Y_hat - Y) / n_sample
        db = 2 * np.sum(Y_hat - Y) / n_sample

        W = W - alpha * dW
        b = b - alpha * db
        
        if i%200==0:
            Y_hat=np.dot(X, W)+b
            L=np.sum((Y-Y_hat)**2)**0.5/n_sample
            print(L,end='\t')

    return W,b

W,b=linear_reg(X_train,Y_train)

10.421461312004459	1.5753299230529596	1.3010925384326066	1.1150379318323123	0.9900455051928722	0.9059846965894982	0.8486191309906469	0.8083911440623579	0.7791472575454574	0.7570256866554005	

## 数据归一化
**Normalization：**
$$
x=\frac{x-x_{min}}{x_{max}-x_{min}}
$$

In [10]:
X=np.row_stack((X_train,X_test))

X_max=X.max(axis=0)
X_min=X.min(axis=0)

X_train_norm=(X_train-X_min)/(X_max-X_min)
X_test_norm=(X_test-X_min)/(X_max-X_min)

对数据归一化之后再测试模型表现：

In [11]:
W,b=linear_reg(X_train_norm,Y_train)

1.1866179843366065	1.1851727557939387	1.1837300851998311	1.1822899688901225	1.1808524032054784	1.1794173844913827	1.1779849090981265	1.1765549733808003	1.1751275736992812	1.1737027064182264	

因为数据做了归一化，整个数据集上的梯度分布得到了改良，所以可以调大学习率，由此可以看出数据标准化在线性回归上的威力：

In [12]:
W,b=linear_reg(X_train_norm,Y_train,alpha=0.1)

0.5898723758342488	0.25669946297613266	0.24193280206822776	0.23824895656765277	0.23642904227502215	0.23526814606885943	0.2344731715988042	0.23391549904818665	0.23351978554054928	0.23323705925155527	

**Standardization：**
$$
x=\frac{x-x_{\mu}}{\sigma}
$$

In [13]:
X=np.row_stack((X_train,X_test))

X_avg=X.mean(axis=0)
X_std=X.std(axis=0)

X_train_std=(X_train-X_avg)/X_std
X_test_std=(X_test-X_avg)/X_std

In [14]:
W,b=linear_reg(X_train_std,Y_train)

1.1533641928430791	1.1528875544037156	1.1524112302061575	1.1519352196779988	1.151459522248928	1.150984137350719	1.150509064417222	1.1500343028843558	1.1495598521900992	1.1490857117744822	

In [15]:
W,b=linear_reg(X_train_std,Y_train,alpha=0.1)

0.9520552989675496	0.23251662877347026	0.2325058668412472	0.23250581106415466	0.23250581077482452	0.2325058107733237	0.23250581077331586	0.23250581077331586	0.23250581077331586	0.23250581077331586	

这个简单实验比较发现，相比于Normalization，Standardization能够更快地加速模型的收敛，这跟最小二乘法对于数据先验分布为正态分布的假设是一致的。

数据归一化工具简单打包：

In [16]:
def Standardization(X_train,X_test):
    X=np.row_stack((X_train,X_test))

    X_avg=X.mean(axis=0)
    X_std=X.std(axis=0)

    X_train_std=(X_train-X_avg)/X_std
    X_test_std=(X_test-X_avg)/X_std
    return X_train_std,X_test_std

## mini-batch梯度下降

In [17]:
def linear_reg(X,Y,alpha=0.000001,max_iter=2000,batch_size=32):
    Y=Y.reshape((-1,1))
    
    n=X.shape[0]
    m=X.shape[1]
    num_batch = n // batch_size
    
    W = np.random.rand(m).reshape((m, 1))  # 权重
    b = 1  # 偏置

    for epoch in range(max_iter):
        
        ######  mini-batch  ######
        for i in range(num_batch + 1):    # 有可能有多余的不完整batch，多循环一次
            start_index = i * batch_size
            end_index = (i + 1) * batch_size
            
            if start_index < n:
                # 切片操作不会引发越界
                X_batch = X[start_index:end_index + 1]
                Y_batch = Y[start_index:end_index + 1]
                
            n_batch=X_batch.shape[0]
            Y_hat_batch=np.dot(X_batch, W)+b

            dW = 2 * X_batch.T.dot(Y_hat_batch - Y_batch) / n_batch
            db = 2 * np.sum(Y_hat_batch - Y_batch) / n_batch
        
        
            W -= alpha * dW
            b -= alpha * db
        ######  mini-batch  ######
        
        if epoch%200==0:
            Y_hat=np.dot(X, W)+b
            L=np.sum((Y-Y_hat)**2)**0.5/n_sample
            print(L,end='\t')

    return W,b

W,b=linear_reg(X_train,Y_train)

1.474682059274926	0.5356777959141976	0.47384792900798556	0.4391595004502859	0.4132333630693115	0.3934206203125686	0.3781957553275647	0.3664223754397032	0.35724210453440275	0.3500104774937606	

单纯的mini-batch并没有很明显的提升模型表现，我们再加上Standardization：

In [18]:
X_train_std,X_test_std=Standardization(X_train,X_test)
W,b=linear_reg(X_train_std,Y_train,batch_size=32)

1.2035063734435572	1.1947475885014098	1.1861772410107605	1.1777872195520669	1.169569794197096	1.1615176004647043	1.1536236237540518	1.1458811842592154	1.1382839223671453	1.130825784539052	

额，表现并不是很理想，甚至在加大max_iter值后模型还是没有收敛，可能是数据量太小，mini-batch引入的随机性对模型的收敛起了一个反作用。